<a href="https://colab.research.google.com/github/emina13/comp_ling/blob/main/3_Iskhakova.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import re
import json
!pip install pymorphy2
import string
import requests

     |████████████████████████████████| 61kB 6.4MB/s 
     |████████████████████████████████| 8.2MB 20.7MB/s 


In [4]:
import pymorphy2


In [25]:
from nltk import word_tokenize


In [6]:
morph = pymorphy2.MorphAnalyzer()

with open("tashkent (1).json") as f:
    init_dict = json.load(f)

titles = list(init_dict.keys())[:-1]
texts = list(init_dict.values())[:-1]

In [9]:
#чищу диакритические символы как в лаб2
def del_diacritics(text):
    text = re.sub(r'Ё', 'Е', text)
    text = re.sub(r'ё', 'е', text)

    text = re.sub(r'а́', 'а', text)
    text = re.sub(r'А́', 'А', text)

    text = re.sub(r'е́', 'е', text)
    text = re.sub(r'É', 'Е', text)

    text = re.sub(r'и́', 'и', text)
    text = re.sub(r'И́', 'И', text)

    text = re.sub(r'о́', 'о', text)
    text = re.sub(r'О́', 'О', text)

    text = re.sub(r'у́', 'у', text)
    text = re.sub(r'У́', 'У', text)

    text = re.sub(r'ы́', 'ы', text)
    text = re.sub(r'Ы́', 'Ы', text)

    text = re.sub(r'я́', 'я', text)
    text = re.sub(r'Я́', 'Я', text)

    text = re.sub(r'ю́', 'ю', text)
    text = re.sub(r'Ю́', 'Ю', text)

    text = re.sub(r'э́', 'э', text)
    text = re.sub(r'Э́', 'Э', text)
    return text

In [10]:
for i in range(len(texts)):
    texts[i] = re.sub(r'\n', ' ', texts[i])  # Удаление символа переноса строки как и в лабе2

In [16]:
while len(re.findall(r'\([^()]*\)', texts[i])) != 0:  # Удаляю лишние скобки и пробелы
        texts[i] = re.sub(r'\([^()]*\)', '', texts[i])
        texts[i] = re.sub(r'\s+(?=(?:[,.?!:;…]))', '', texts[i])
        texts[i] = re.sub(r'\s+', ' ', texts[i])
    #заменяем диакритические символы
texts[i] = del_diacritics(texts[i])

In [17]:
main_dict = dict(zip(titles, texts))
with open("tashkent_dict.json", "w") as f:
    #корпус в виде словаря в формате json
    json.dump(main_dict, f, ensure_ascii=False)

In [19]:
with open("corpus.json") as f:  # корпус подготовленный в лабе2
    corpus = json.load(f)
    while len(re.findall(r'\([^()]*\)', corpus)) != 0:
        corpus = re.sub(r'\([^()]*\)', '', corpus)
        corpus = re.sub(r'\s+(?=(?:[,.?!:;…]))', '', corpus)
        corpus = re.sub(r'\s+', ' ', corpus)
    del_diacritics(corpus)

In [23]:
def no_punct(tokens_list):  #избавляюсь от пунктуации
    punct = string.punctuation + "—" + "«" + "»"
    return [token.lower() for token in tokens_list if token not in punct]

In [29]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [30]:
tokens = word_tokenize(corpus)
tokens = no_punct(tokens)

In [31]:
normed_tokens = [morph.parse(i)[0].normal_form for i in tokens  # Список начальных форм всех токенов
                 if str.isalpha(morph.parse(i)[0].normal_form)] # print(normed_tokens)
unique_norm_tokens = list(set(normed_tokens))

In [32]:
with open("uniques_corpus.json", "w") as f: #корпус в виде словаря в json
    json.dump(unique_norm_tokens, f, ensure_ascii=False)

In [34]:
normed_dict = {}
md_keys = list(main_dict.keys())
md_values = list(main_dict.values())
for i in range(len(main_dict)):
    tokened_value = md_values[i]
    tokened_value = word_tokenize(tokened_value)
    tokened_value = no_punct(tokened_value)
    normed_tokens = [morph.parse(i)[0].normal_form for i in tokened_value
                     if str.isalpha(morph.parse(i)[0].normal_form)]
    normed_dict[md_keys[i]] = normed_tokens

In [35]:
with open("norms_dict.json", "w") as f:
    json.dump(normed_dict, f, ensure_ascii=False)

In [36]:
with open("uniques_corpus.json") as f:
    unique_norm_tokens = json.load(f)

with open("norms_dict.json") as f:
    normed_dict = json.load(f)

In [38]:
word_usage = {}
for word in unique_norm_tokens:
    counter = 0
    docs_include_word = {}
    list_docs_include_word = []
    for i in range(len(normed_dict)):
        if word in list(normed_dict.values())[i]: #Номер документа, i
            c = list(normed_dict.values())[i].count(word)
            counter += c
            list_docs_include_word.append(list(normed_dict.keys())[i])
    docs_include_word[counter] = list_docs_include_word
    word_usage[word] = docs_include_word
with open("tokens_appearance.json", "w") as f:
    json.dump(word_usage, f, ensure_ascii=False)

In [39]:
with open("tokens_appearance.json") as f:
    word_usage = json.load(f)

print(word_usage)

{'взорваться': {'2': ['Теракт в Ташкенте 16 февраля 1999 года', 'Террористический акт в Ташкенте 16 февраля 1999 года']}, 'немало': {'3': ['Аквапарк', 'Большевики', 'Карши']}, 'отрицательный': {'2': ['Высота над уровнем моря', 'Часовой пояс']}, 'трактор': {'1': ['Ташкентский тракторный завод']}, 'нагнетание': {'1': ['Аквапарк']}, 'ануштегинид': {'2': ['Ануштегиниды', 'Хорезмшах']}, 'густо': {'1': ['Тугаи']}, 'ныне': {'15': ['Азия', 'Большая советская энциклопедия', 'Казахстан', 'Конституционный суд Республики Узбекистан', 'Оксарой', 'Система административного управления Туркестаном', 'Средняя Азия', 'Ташкентский институт текстильной и лёгкой промышленности', 'Температура', 'Уйгуры', 'Шкала Бофорта', 'Шымкент']}, 'садовый': {'1': ['Японский сад']}, 'форма': {'31': ['Австралия', 'Армения', 'Богослов', 'Бруней', 'Великобритания', 'Восточный Тимор', 'Вьетнам', 'Государственный строй Узбекистана', 'Грузия', 'Еврейский язык', 'Китайский язык', 'Краткая еврейская энциклопедия', 'Мусульмане', 